In [1]:
import pennylane as qml
import numpy as np
import random
import os
import pandas as pd
import torch
from torch.autograd import Variable

In [2]:
np.random.seed(42)

qubits = 8

layer = 1

batch_size = 512

epoch = 5000

z_dim = 8

lr = 0.04

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def sample_z(sbatch_size):
    """Sample the random noise"""
    return np.random.normal(0, 1, size=(batch_size, z_dim))

In [5]:
dev = qml.device('default.qubit', wires=qubits)
@qml.qnode(dev, interface='torch', diff_method='backprop')
def gen_circuit(w):
    # random noise as generator input
    z1 = random.uniform(-1, 1)
    z2 = random.uniform(-1, 1)
    
    # construct generator circuit for both atom vector and node matrix
    for i in range(qubits):
        qml.RY(np.arcsin(z1), wires=i)
        qml.RZ(np.arcsin(z2), wires=i)
    for l in range(layer):
        for i in range(qubits):
            qml.RY(w[i], wires=i)
        for i in range(qubits-1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(w[i+qubits], wires=i+1)
            qml.CNOT(wires=[i, i+1])
    return [qml.expval(qml.PauliZ(i)) for i in range(qubits)]

In [6]:
gen_weights = torch.tensor(list(np.random.rand(layer*(qubits*2-1))*2*np.pi-np.pi), requires_grad=True)

In [7]:
best_params = torch.tensor(list(np.random.rand(layer*(qubits*2-1))*2*np.pi-np.pi), requires_grad=True)

In [8]:
opt = torch.optim.RMSprop([gen_weights], lr)

In [9]:
best_cost = 2.0

In [10]:
for n in range(epoch):
    opt.zero_grad()
    loss = torch.nn.MSELoss()
    z = sample_z(batch_size)
    z1 = torch.from_numpy(z).to(device).float()
    sample_list = [gen_circuit(gen_weights) for i in range(batch_size)]
    z2 = torch.stack(tuple(sample_list)).to(device).float()
    output = loss(z1, z2)
    output.backward()
    opt.step()
    
    if output < best_cost:
        best_cost = output
        best_params = gen_weights
        print(n, best_cost)
    

/home/ken/anaconda3/envs/molgan-pt/lib/python3.7/site-packages/torch/autograd/__init__.py:149: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/Copy.cpp:240.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


0 tensor(1.4393, device='cuda:0', grad_fn=<MseLossBackward>)
1 tensor(1.3430, device='cuda:0', grad_fn=<MseLossBackward>)
2 tensor(1.2328, device='cuda:0', grad_fn=<MseLossBackward>)
4 tensor(1.2304, device='cuda:0', grad_fn=<MseLossBackward>)
5 tensor(1.1924, device='cuda:0', grad_fn=<MseLossBackward>)
12 tensor(1.1864, device='cuda:0', grad_fn=<MseLossBackward>)
16 tensor(1.1790, device='cuda:0', grad_fn=<MseLossBackward>)
30 tensor(1.1553, device='cuda:0', grad_fn=<MseLossBackward>)
294 tensor(1.1301, device='cuda:0', grad_fn=<MseLossBackward>)
310 tensor(1.1299, device='cuda:0', grad_fn=<MseLossBackward>)


KeyboardInterrupt: 

In [13]:
import csv
with open(os.path.join('molgan_red_weights.csv'), 'a') as file:
    writer = csv.writer(file)
    writer.writerow([str(310)]+list(best_params.detach().numpy()))